<a href="https://colab.research.google.com/github/spencer18001/machine-learning-zoomcamp/blob/main/09/hw_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install tensorflow=2.17.1 pillow
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras

In [2]:
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image
import requests
import tensorflow as tf
from tensorflow import keras

In [3]:
model = keras.models.load_model("model_2024_hairstyle.keras")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model_2024_hairstyle.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpupr4r2vs'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  137818503311984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137818503267424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137818503275168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137818503310224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137818503315152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137818503312688: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [4]:
!ls -lh model_2024_hairstyle.tflite # Q1

-rw-r--r-- 1 root root 77M Dec  7 18:15 model_2024_hairstyle.tflite


In [5]:
interpreter = tf.lite.Interpreter(model_path="model_2024_hairstyle.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
output_index # Q2

13

In [6]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = img.resize(target_size, Image.NEAREST)
    return img

In [7]:
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
img = download_image(url)
img = prepare_image(img, target_size=(200, 200))

In [8]:
img_array = np.array(img, dtype="float32") / 255.0
img_array[0, 0, 0] # Q3

0.23921569

In [12]:
interpreter.set_tensor(input_index, [img_array])
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
float(preds[0, 0]) # Q4

0.8937740921974182

***This cell is running in the GitHub Codespace environment.***

```shell
docker pull agrigorev/model-2024-hairstyle:v3
docker images agrigorev/model-2024-hairstyle:v3 --format "{{.Size}}"
```

output:
```shell
782MB # Q5
```

***This cell is running in the GitHub Codespace environment.***

```shell
docker build -t hairstyle-lambda .
docker run -it --rm -p 8080:8080 hairstyle-lambda
```

In [10]:
# this url is copied from GitHub Codespace's session
url = "https://psychic-goldfish-x5994xpgpgw5hxg5-8080.app.github.dev/"+"2015-03-31/functions/function/invocations"
data = {"url": "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"}
requests.post(url, json=data).json() # Q6

0.42985352873802185